In [75]:
import pandas as pd
import numpy as np
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [76]:
df = pd.read_csv(
    r"C:\Users\syngu\OneDrive\Documents\GitHub\HandGestureWithLSTM\models\data_main\concated1.csv"
)
window = 60

In [77]:
df["label"].value_counts()

label
thirsty           7448
name              7159
can               6984
morning           6731
help              6716
eat               6706
you               6560
good              6132
please            6083
ILY2              5015
no                4882
I                 4714
sorry             4630
hungry            4109
drink             3982
you're welcome    3425
thanks            3031
my                2557
Yes               2479
yes               2200
Name: count, dtype: int64

In [78]:
labels = df["label"].unique().tolist()
for label in labels:
    sample_size = len(df.loc[df["label"] == label]) //window *window
    df.loc[df["label"] == label] = df.loc[df["label"] == label].head(
        sample_size
    )

In [79]:
len(df["label"].value_counts())

20

In [80]:
encoder = LabelEncoder()

X_sequence = []
y_sequence = []
for label in df["label"].unique().tolist():
    temp_df = df.loc[df["label"]==label]
    temp_X = temp_df.drop(columns=["label"])
    for i in range(len(temp_df)-window):
        X_sequence.append(temp_X[i:i+window])
        label = temp_df['label'].iloc[i+window]
        y_sequence.append(label)


X = np.array(X_sequence)
y = np.array(y_sequence)

y = encoder.fit_transform(y)
y = to_categorical(y,len(df["label"].value_counts()))

In [81]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [82]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((89856, 60, 63), (89856, 20), (9984, 60, 63), (9984, 20))

In [83]:
encoder.classes_

array(['I', 'ILY2', 'Yes', 'can', 'drink', 'eat', 'good', 'help',
       'hungry', 'morning', 'my', 'name', 'no', 'please', 'sorry',
       'thanks', 'thirsty', 'yes', 'you', "you're welcome"], dtype='<U14')

In [84]:
model = Sequential(
    [
        # InputLayer((30,63)),
        InputLayer((60, 63)),
        LSTM(128, return_sequences=True),
        LSTM(128, return_sequences=False),
        Dense(64, activation="relu"),
        Dense(32,activation='relu'),
        Dense(y.shape[1], activation="softmax"),
    ]
)

early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=10, restore_best_weights=True
)

model.compile(
    loss="categorical_crossentropy", metrics=["accuracy"], optimizer=Adam(0.001)
)
# model = tf.keras.models.load_model(
#     r"C:\Users\syngu\OneDrive\Documents\GitHub\HandGestureWithLSTM\models\test_model_LSTM1.keras"
# )

In [85]:
model.fit(
    X_train,
    y_train,
    epochs=999,
    validation_split=0.01,
    batch_size=512,
    callbacks=[early_stopping],
)

Epoch 1/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 218ms/step - accuracy: 0.2195 - loss: 2.4579 - val_accuracy: 0.5740 - val_loss: 1.1856
Epoch 2/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 37s 214ms/step - accuracy: 0.7309 - loss: 0.7790 - val_accuracy: 0.9021 - val_loss: 0.2630
Epoch 3/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 39s 227ms/step - accuracy: 0.7627 - loss: 0.7881 - val_accuracy: 0.9055 - val_loss: 0.3054
Epoch 4/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 39s 223ms/step - accuracy: 0.9136 - loss: 0.2543 - val_accuracy: 0.9644 - val_loss: 0.1277
Epoch 5/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 229ms/step - accuracy: 0.9567 - loss: 0.1357 - val_accuracy: 0.9755 - val_loss: 0.0847
Epoch 6/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 39s 222ms/step - accuracy: 0.9698 - loss: 0.0968 - val_accuracy: 0.9822 - val_loss: 0.0455
Epoch 7/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 38s 221ms/step - accuracy: 0.9776 - loss: 0.0720 - val_accuracy: 0.9766 - val_loss: 0.0687
Epoch 8/999
174/174 ━━━━━━━━━━━━━━━━━━━━ 38s 221ms/step - accuracy: 0.9448 -

In [86]:
model.evaluate(X_test, y_test)

312/312 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9985 - loss: 0.0065


[0.0067401123233139515, 0.9987980723381042]

In [87]:
model.save("test_model_LSTM3.keras")